In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random
from copy import deepcopy

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [8]:
clients, test_x, test_y = utils.set_data(selected_model, True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [32]:
f1_before = []
for client in clients:
    client.init_empty_model(0.01)
    client.train_model(epochs=50)
    try: 
        score = client.test_model_f1(y_test=test_y, X_test=test_x)
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

0.6984561752988048
0.09834562653665042
0.09983713010554816
0.0
0.000846740050804403


In [33]:
fedavg = Fedavg("global", 0.1, selected_model)

Waitiing for a Connection..


In [34]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True
epochs = 2

for client in clients:
    client.init_empty_model(0.01)
    X_train, X_test, y_train, y_test = client.split_data(0.96)

    # X_train = client.x_chunks[0]
    # y_train = client.y_chunks[0]

    client.train_model(X_train,y_train,epochs=epochs)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.init_global_model(client.model)
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.
.
.


In [35]:
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.0


In [36]:
number_of_rounds = 24
epochs = 2
max_score = 0
optimal_model = None


for round in range(number_of_rounds):
    print(round, end=' ')

    applicable_clients = random.sample((clients), len(clients))#random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        print(f'.', end='')

        X_train, X_test, y_train, y_test = client.split_data(0.98)

        # X_train = client.x_chunks[round + 1]
        # y_train = client.y_chunks[round + 1]

       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        client.load_global_model(fedavg.model)
        client.train_local_agent(X_train, y_train, epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

0 .....0.04871001031991744
1 .....0.11807833301176926
2 .....0.3520450405696307
3 .....0.6156721399012148
4 .....0.69222235917663
5 .....0.6977707394323304
6 .....0.6982894577217031
7 .....0.7000971345313259
8 .....0.7034197395643179
9 .....0.7066861883845779
10 .....0.7241087613293051
11 .....0.738788831137435
12 .....0.7546262917567893
13 .....0.7655180693960859
14 .....0.7676428228516563
15 .....0.7782138185277678
16 .....0.7879933821791538
17 .....0.7913464948575482
18 .....0.8042105263157895
19 .....0.8114897081055936
20 .....0.8165073443693168
21 .....0.8264058679706601
22 .....0.8368548666435747
23 .....0.8404292142609899


In [30]:
# x = test_x
# y_hat = optimal_model.predict(x)
# score = f1_score(test_y,y_hat)
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)
fedavg.model = optimal_model
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.7817234274633378
0.7817234274633378


In [31]:
for i in f1_before:
    print(f'Difference {score-i}')

Difference 0.07948893085274267
Difference 0.6834310492875777
Difference 0.6830168599214869
Difference 0.7817234274633378
Difference 0.7817234274633378


## Centralized Zone

In [14]:
client1 = Client("node1","0.0.0.0", 5001, selected_model)
client1.x = clients[0].x
client1.init_empty_model(0.01)


for client in clients:
    client1.train_model(x=client.x,y=client.y,epochs=50)
    print(f'.')

Creating node1.
.
.
.
.
.


In [15]:
score = client1.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.7945415125621731
